# data merging 

_desired model inputs:_

Player Efficiency Ratings of each player - seasons_per_s.csv 

Location of the game- arena volume_game_stats.parquet.gzip

Teams average last season then (last 5 games) - subset
Individual game stats (2022)

potentially compare past matchups between the two teams 



In [2]:
import pandas as pd

In [15]:
cd ..

/Users/bryce/Desktop/Applied Machine Learning /project-checkpoint-1-nba-prediction/dev


In [5]:
season_schedules = pd.concat([pd.read_parquet('[2015, 2016].parquet.gzip'),pd.read_parquet('[2017, 2018].parquet.gzip'), pd.read_parquet('[2019, 2020].parquet.gzip'), pd.read_parquet('[2021, 2022].parquet.gzip'), pd.read_parquet("[2022, 2023, 2024].parquet.gzip")], ignore_index = True)

season_schedules['date'] = pd.to_datetime(season_schedules['date'])
season_schedules.to_csv("season_schedules.csv")

In [9]:
season_game_stats = pd.concat([pd.read_parquet('[2015, 2016]_game_stats.parquet.gzip'),pd.read_parquet('[2017, 2018]_game_stats.parquet.gzip'), pd.read_parquet('[2019, 2020]_game_stats.parquet.gzip'), pd.read_parquet('[2021, 2022]_game_stats.parquet.gzip'), pd.read_csv("[2022, 2023, 2024]_game_stats.csv")], ignore_index = True)
season_game_stats['date'] = pd.to_datetime(season_game_stats['date'])
season_game_stats = season_game_stats.drop(columns=['Unnamed: 0'])
season_game_stats.to_csv("combined_game_stats.csv")

In [10]:
all_season_rosters = pd.read_csv('season_rosters.csv')
# all_season_rosters["team_id"] = all_season_rosters["team_dir"].str[7:10]
all_season_rosters

,Unnamed: 0,Year,Team,team_dir,Player,player_dir
0,0,2015,Golden State Warriors,/teams/GSW/2015.html,Leandro Barbosa,/players/b/barbole01.html
1,1,2015,Golden State Warriors,/teams/GSW/2015.html,Harrison Barnes,/players/b/barneha02.html
2,2,2015,Golden State Warriors,/teams/GSW/2015.html,Andrew Bogut,/players/b/bogutan01.html
3,3,2015,Golden State Warriors,/teams/GSW/2015.html,Stephen Curry,/players/c/curryst01.html
4,4,2015,Golden State Warriors,/teams/GSW/2015.html,Festus Ezeli,/players/e/ezelife01.html
...,...,...,...,...,...,...
5948,5948,2024,Charlotte Hornets,/teams/CHO/2024.html,Mark Williams,/players/w/willima07.html
5949,5949,2024,Charlotte Hornets,/teams/CHO/2024.html,Aleksej Pokusevski,/players/p/pokusal01.html
5950,5950,2024,Charlotte Hornets,/teams/CHO/2024.html,NaN,/players/b/baileam01.html
5951,5951,2024,Charlotte Hornets,/teams/CHO/2024.html,NaN,/players/b/boldema01.html


In [98]:
# player_pers = pd.read_csv('seasons_per_s.csv')
player_pers['Year'] = player_pers['season']
player_pers['Year'] = player_pers['Year'].str[:2] + player_pers['Year'].str[-2:]
player_pers['Year'] = player_pers['Year'].astype('int64')
player_pers

,season,team_id,player_dir,per,Year
0,2014-15,GSW,/players/b/barbole01.html,15.3,2015
1,2015-16,GSW,/players/b/barbole01.html,11.7,2016
2,2016-17,PHO,/players/b/barbole01.html,11.5,2017
3,2014-15,GSW,/players/b/barneha02.html,13.4,2015
4,2015-16,GSW,/players/b/barneha02.html,12.3,2016
...,...,...,...,...,...
5138,2021-22,POR,/players/b/browngr01.html,11.0,2022
5139,2021-22,POR,/players/c/cumbeja01.html,8.4,2022
5140,2021-22,POR,/players/m/mcgrica01.html,12.5,2022
5141,2021-22,POR,/players/w/watfotr01.html,15.8,2022


In [102]:
# roster merge 
player_info_per = all_season_rosters.merge(player_pers, how = "right", on=["Year", "player_dir", "team_id"])
player_info_per
# player_info_per.to_parquet("player_info_per.parquet.gzip")

,Year,Team,team_dir,Player,player_dir,team_id,season,per
0,2015,Golden State Warriors,/teams/GSW/2015.html,Leandro Barbosa,/players/b/barbole01.html,GSW,2014-15,15.3
1,2016,Golden State Warriors,/teams/GSW/2016.html,Leandro Barbosa,/players/b/barbole01.html,GSW,2015-16,11.7
2,2017,Phoenix Suns,/teams/PHO/2017.html,Leandro Barbosa,/players/b/barbole01.html,PHO,2016-17,11.5
3,2015,Golden State Warriors,/teams/GSW/2015.html,Harrison Barnes,/players/b/barneha02.html,GSW,2014-15,13.4
4,2016,Golden State Warriors,/teams/GSW/2016.html,Harrison Barnes,/players/b/barneha02.html,GSW,2015-16,12.3
...,...,...,...,...,...,...,...,...
5138,2022,Portland Trail Blazers,/teams/POR/2022.html,Greg Brown III,/players/b/browngr01.html,POR,2021-22,11.0
5139,2022,Portland Trail Blazers,/teams/POR/2022.html,Jarron Cumberland,/players/c/cumbeja01.html,POR,2021-22,8.4
5140,2022,Portland Trail Blazers,/teams/POR/2022.html,Cameron McGriff,/players/m/mcgrica01.html,POR,2021-22,12.5
5141,2022,Portland Trail Blazers,/teams/POR/2022.html,Trendon Watford,/players/w/watfotr01.html,POR,2021-22,15.8


In [8]:
# def get_team_stats(date, num_games, team_id): #maybe add stats data as a param 
#     #to prevent data leakage drop games newer than the date of the game 
#     all_game_stats = pd.read_parquet("[2021, 2022]_game_stats.parquet.gzip")
#     all_game_stats= all_game_stats[all_game_stats['date'] < date
#     team_game_stats = 
def get_roster(all_season_rosters):
    


In [63]:
pd.read_parquet("[2015, 2016].parquet.gzip")[pd.read_parquet("[2015, 2016].parquet.gzip")['season']==2015]

,season,date,away_team_id,home_team_id,arena,link
0,2015,"Tue, Oct 28, 2014",ORL,NOP,Smoothie King Center,/boxscores/201410280NOP.html
1,2015,"Tue, Oct 28, 2014",DAL,SAS,AT&T Center,/boxscores/201410280SAS.html
2,2015,"Tue, Oct 28, 2014",HOU,LAL,STAPLES Center,/boxscores/201410280LAL.html
3,2015,"Wed, Oct 29, 2014",MIL,CHO,Time Warner Cable Arena,/boxscores/201410290CHO.html
4,2015,"Wed, Oct 29, 2014",PHI,IND,Bankers Life Fieldhouse,/boxscores/201410290IND.html
...,...,...,...,...,...,...
8605,2015,"Wed, Apr 15, 2015",DET,NYK,Madison Square Garden (IV),/boxscores/201504150NYK.html
8606,2015,"Wed, Apr 15, 2015",MIA,PHI,Wells Fargo Center,/boxscores/201504150PHI.html
8607,2015,"Wed, Apr 15, 2015",IND,MEM,FedEx Forum,/boxscores/201504150MEM.html
8608,2015,"Wed, Apr 15, 2015",DEN,GSW,Oracle Arena,/boxscores/201504150GSW.html


In [4]:
pd.read_parquet("[2022, 2023, 2024].parquet.gzip")

,season,date,away_team_id,home_team_id,arena,link
0,2022,"Tue, Oct 19, 2021",BRK,MIL,Fiserv Forum,/boxscores/202110190MIL.html
1,2022,"Tue, Oct 19, 2021",GSW,LAL,Crypto.com Arena,/boxscores/202110190LAL.html
2,2022,"Wed, Oct 20, 2021",IND,CHO,Spectrum Center,/boxscores/202110200CHO.html
3,2022,"Wed, Oct 20, 2021",CHI,DET,Little Caesars Arena,/boxscores/202110200DET.html
4,2022,"Wed, Oct 20, 2021",BOS,NYK,Madison Square Garden (IV),/boxscores/202110200NYK.html
...,...,...,...,...,...,...
2998,2024,"Sun, Mar 31, 2024",DAL,HOU,Toyota Center,/boxscores/202403310HOU.html
2999,2024,"Sun, Mar 31, 2024",CHI,MIN,Target Center,/boxscores/202403310MIN.html
3000,2024,"Sun, Mar 31, 2024",OKC,NYK,Madison Square Garden (IV),/boxscores/202403310NYK.html
3001,2024,"Sun, Mar 31, 2024",GSW,SAS,Frost Bank Center,/boxscores/202403310SAS.html
